# 2. Extraction-based MRC

## 강의소개

- 2강에서는 추출기반으로 기계독해를 푸는 방법에 대해 알아보겠습니다.
- 추출기반으로 기계독해 문제에 접근한다는 것의 의미를 이해하고, 실제 추출기반 기계독해를 어떻게 풀 수 있을지에 대해 배워볼 예정입니다.
- 학습 전 준비해야할 단계와 모델 학습 단계, 그리고 추출기반으로 얻어낸 답을 원하는 텍스트의 형태로 변형하는 방법에 대해 이번 강의에서 자세히 알아보겠습니다.

<br>

## Further Reading

- [SQuAD 데이터셋 둘러보기](https://rajpurkar.github.io/SQuAD-explorer/)
- [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805)
- [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning](http://jalammar.github.io/illustrated-bert/))
- [Huggingface datasets](https://huggingface.co/datasets)

<br>

## 2.1 Extraction-based MRC

- Extraction-based MRC의 정의
- Extraction-based MRC의 평가 방법
- Extraction-based MRC Overview

<br>

### 2.1.1 Extraction-based MRC 문제 정의

- 질문(question)의 답변(answer)이 항상 주어진 지문(context)내에 span으로 존재
- e.g. SQuAD, KorQuAD, NewsQA, Natural Questions, etc.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1jnEjDa2KOaTe3SMBbFLJZETnlAt5t18L' width=800/>

<br>

- 이러한 데이터셋을 웹사이트에서 직접 다운로드 받는 것이 아닌 huggingface의 `datasets` 라이브러리를 통해 다운로드 받을 수 있다.
- 이를 이용하면 일관된 포멧의 데이터셋을 사용할 수 있다.
- Extraction-based MRC datasets in HuggingFace datasets
- [https://huggingface.co/datasets](https://huggingface.co/datasets)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1jop85XTH47WM7NSPb1xO_RBfFnzbEb8u' width=800/>

<br>

### 2.1.2 Extraction-based MRC 평가 방법

- SQuAD 데이터셋의 리더보드와 KorQuAD 데이터셋의 리더보드

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1jpUehUJUagDo4DejrANTo4i1VvpuVMK3' width=800/>

<br>

#### 2.1.2.1 Exact Match(EM) Score

- 예측값과 정답이 캐릭터 단위로 완전히 똑같을 경우에만 1점 부여
- 하나라도 다른 경우 0점

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1jvcF7dA2to9D_bJQtz6dKIb830Ap3ki9' width=800/>

<br>

#### 2.1.2.2 F1 Score

- 예측값과 정답의 overlap을 비율로 계산
- 0점과 1점 사이의 부분 점수를 받을 수 있음

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1jz9PnDPWL_VTGjaPRvAUmGldVk2zcY8w' width=600/>

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kFjyOOL0hOydykRNh7cORCLTRrKqXAoX' width=250/>

- Precision
  - 예측값의 총 토큰 수에 대한 예측값과 실제값의 겹치는 토큰 수의 비율
  - 필요없이 예측값이 길어지게 되면 겹치는 토큰이 많더라도 분모값이 커지기 때문에 precision이 작아진다.
- Recall
  - 실제값의 총 토큰 수에 대한 예측값과 실제값의 겹치는 토큰 수의 비율
  - 겹치는 토큰 수가 적으면 recall이 작아진다.

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kGNoSo8hpKlr7UlYc3Ix0qLRDgG6NXkg' width=800/>

- 모든 ground truth에 대해 f1 score를 각각 계산하여 그 중 가장 높은 값을 내보내는 방식으로 evaluation을 한다.

<br>

### 2.1.3 Extraction-baesd MRC Overview

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kGQsa7Fh_5bSzKs2SC67adV--66ncOud' width=800/>

Step 1: Pre-process

- context와 question, answer가 입력으로 주어진다.
- answer는 input 보다는 학습할 때 필요한 데이터이다.
- context와 question이 토크나이징 되어 모델의 입력으로 들어간다.

<br>

Step 2: Fine-tuning

- 이 토크나이징된 값들을 word embedding을 통해 벡터화한다.
- 벡터들이 모델의 입력으로 들어간다.

<br>

Step 3: Post-processing

- 모델이 내보내는 값은 시작점의 위치와 끝점의 위치이다.
  - 이때 위치의 인덱스를 내보내는 것이 아니다.
  - 모델의 출력으로 context와 question의 각 토큰에 해당하는 contextualize vector들이 나온다.
  - 각각의 벡터를 scale up value로 내보냄으로서 각 토큰들이 시작점과 끝점이 될 수 있는 점수를 얻고 가장 점수가 높은 토큰을 찾는 방식으로 start position과 end position을 예측하게 된다.
- 이렇게 구해진 start position과 end position을 가지고 각 position 사이에 있는 span을 예측 결과로 내보내게 된다.


<br>

## 2.2 Pre-processing

- 입력 예시
- Tokenization
- Special tokens, attention mask, and token ids for BERT
- 출력 형태

<br>

### 2.2.1 입력 예시

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kH7-Ymh4nl_Yy_RXVlOTiKiZJoAK7pjy' width=800/>

<br>

### 2.2.2 Tokenization

- 텍스트를 작은 단위(Token)로 나누는 것
- 띄어쓰기 기준, 형태소, subword 등 여러 단위 토큰 기준이 사용됨
- 최근엔 Out-Of-Vocabulary(OOV) 문제를 해결해주고 정보학적으로 이점을 가진 Byte Pair Encoding(BPE)을 주로 사용함
- 본강에선 BPE 방법론 중 하나인 WordPiece Tokenizer를 사용

- WordPiece Tokenizer 사용 예시
  - `"미국 군대 내 두번째로 높은 직위는 무엇인가?"`
  - `['미국', '군대', '내', '두번째', '##로', '높은', '직', '##위는', '무엇인가', '?']`

- WordPiece에서는 자주 나오는 단어들은 하나로 이루어지고 비교적 자주 나오지 않는 단어들은 따로 나눠지는 것을 볼 수 있다.
  - `직위는`라는 단어는 자주 등장하지 않으므로 `직`과 `##위는`로 분리되어 토크나이징된다.

<br>

### 2.2.3 Special Tokens

- Special token인 `[SEP]`을 이용하여 question과 context를 구분하도록 한다.



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kKV86NpPw0kFjLg1K9pCFkjZ-7Ndk63Q' width=800/>

- 입력 표현 결과

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kKiePETKHGS-TMrbW3gshm7KQSmaGWRF' width=800/>

- 실제 토큰화된 결과
  - special token들은 토크나이징 이후에도 계속 존재한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kSDxbG9o4bA8DQlfy3_t0v2LGnGg4owU' width=800/>

<br>

### 2.2.4 Attention Mask

- 입력 시퀀스 중에서 attention을 연산할 때 무시할 토큰을 표시
- 0은 무시, 1은 연산에 포함
- 보통 `[PAD]`와 같은 의미가 없는 특수 토큰을 무시하기 위해 사용
- question에 해당하는 토큰들의 attention mask 값은 1로 되어 있다.
  - 질문에서 정답과 관련된 정보를 찾아내야 하므로

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kYfMGQk2uV6QHqCuKWFS25oNUuxKnew3' width=800/>

<br>

### 2.2.5 Token Type IDs

- 토크나이징할 때 `[SEP]` 스페셜 토큰을 이용하여 question과 context를 구분했다.
- 입력이 2개 이상의 시퀀스일 때(ex. 질문 & 지문), 각각에게 ID를 부여한 token type ids를 사용하여 모델이 구분해서 해석하도록 유도한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kZ4x9cCqG9nEtp0Gb-Iil3Rnq53wNzwk' width=800/>

<br>

### 2.2.6 모델 출력값

- 정답은 문서 내 존재하는 연속된 단어 토큰(span)이므로, span의 시작과 끝 위치를 알면 정답을 맞출 수 있다.
- Extraction-based에선 답안을 생성하기 보다, **시작위치와 끝위치를 예측**하도록 학습함
- 즉, Token Classification 문제로 치환

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kbYAkP6zfaiW-fyakXVpwaNgFYDcPDUT' width=800/>

- 정답은 text-level에서 character 위치로 파악하고 있지만 토크나이징 한 후 학습 후 모델의 출력이 해당 토큰이 정답 토큰의 시작 위치 또는 끝 위치에 해당하는 지를 알려주기 때문에 이를 처리하기 위한 작업이 필요하다.

<br>

## 2.3 Fine-tuning

- Model
- Fine-tuning

<br>

### 2.3.1 Extraction-based MRC Overview

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kpRoPKxfrvufJ2wHYtChKsh07D7qTlHK' width=800/>

- 모델의 입력으로 context와 question의 토큰들의 임베딩 벡터들이 주어진다.
- 모델의 출력 또한 context와 question의 토큰들의 임베딩 벡터들이 나오지만 입력과는 다르게 contextualized 임베딩 벡터들이고 활용도가 높다.
- MRC task에 fine-tuning하기 위해서는 출력으로 나온 contextualized embedding vector들을 점수화시켜야 한다.


<br>

### 2.3.2 Fine-tuning BERT

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1ktvMCcIfzmaqCVc1UrP4fjWgH8IKbzct' width=800/>

- context 내에서 정답에 해당하는 토큰의 각 임베딩 벡터들을 linear transformation을 통해서 각 토큰마다 하나의 숫자들이 나올 수 있도록 바꿔준다.
  - 이 linear transformation 또한 학습 대상이 된다.
- 이렇게 되면 각각의 출력 토큰마다 하나의 숫자가 나오게 되고 해당 숫자를 점수로 볼 수 있다.
  - 값이 클수록 높은 점수로 볼 수 있다.
- start position과 end position을 위의 방법으로 각각 구하여 최종 예측값을 생성한다.
- 학습할 때는 이 수치를 softmax를 적용하여 Negative Log Likelihood로 학습하는 방식을 취한다.

<br>

## 2.4 Post-processing

- 불가능한 답 제거하기
- 최적의 답안 찾기

<br>

### 2.4.1 불가능한 답 제거하기

- 다음과 같은 경우 candidate list에서 제거
  - End position이 start position보다 앞에 있는 경우 (e.g., start=90, end=80)
  - 예측한 위치가 context를 벗어난 경우 (e.g., question 위치쪽에 답이 나온 경우)
  - 미리 설정한 max_answer_length보다 길이가 더 긴 경우

<br>

### 2.4.2 최적의 답안 찾기

1. Start/end position prediction에서 score(logits)가 가장 높은 N개를 각각 찾는다.
2. 불가능한 start/end 조합을 제거한다.
3. 가능한 조합들을 score의 합이 큰 순서대로 정렬한다.
4. Score가 가장 큰 조합을 최종 예측으로 선정한다.
5. Top-k가 필요한 경우 차례대로 내보낸다.